# 6.3. Padding and Stride

In [1]:
import torch
from torch import nn

- padding은 보통 kernel size에 의해 data의 size가 작아지는 것을 방지하기 위하여 input에 0을 포함하는 row나 column을 추가합니다. 
- stride는 kernel이 이동하는 간격을 바꾸어 주어 data의 size를 줄여주는 데에 사용될 수 있습니다.
- nn.Conv2d에 대해서 padding과 stride 모두 두 축에 대해 각각 지정해 줄 수 있습니다. scalar로 입력할 경우 두 축에 대해 같은 값을 적용하게 됩니다.

In [2]:
conv2d = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(3,3),stride=(1,1),padding=(1,1),bias=False)
#conv2d.weight.data[:]=torch.tensor([[1,0,-1],[2,0,-2],[1,0,1]])
X = torch.ones(size=(8, 8))
Y = conv2d(X.reshape(1,1,8,8))
Y.shape

torch.Size([1, 1, 8, 8])

- nn.Conv2d의 파라미터들을 명시하지 않고 숫자만 입력하여도 되지만, 순서를 주의하여야 합니다. 파라미터의 이름을 명시할 경우 순서를 바꾸어도 괜찮으나, 파라미터의 이름을 명시하지 않는 경우에는 반드시 정해진 순서대로 숫자들을 입력하여야 합니다.

In [3]:
conv2d = nn.Conv2d(1,1,3,1,1,bias=False)
X = torch.ones(size=(8, 8))
Y = conv2d(X.reshape(1,1,8,8))
Y.shape

torch.Size([1, 1, 8, 8])

# 6.4. Multiple Input and Output Channels

- nn.Conv2d에서 multiple input/output은 채널 수를 정의해 줌으로써 간단하게 구현할 수 있습니다.
- nn.Conv2d는 지정해준 채널 수에 맞게 convolutional layer에 필요한 kernel의 shape를 결정해 줍니다.
- 이 때 nn.Conv2d의 input은 지정해준 채널 수와 맞아야 합니다. 
- PyTorch에서 data는 (batch, channel, data)의 순서로 차원이 지정됩니다. 
- 예를 들어 batch_size = 4, input_channel = 3, 64x64의 2d data라고 하면 (4,3,64,64)의 형태로 input data가 형성되어야 합니다.

In [4]:
conv2d = nn.Conv2d(3,2,3,1,1,bias=False)
#conv2d = nn.Conv2d(in_channels=3,out_channels=2,kernel_size=3,stride=1,padding=1,bias=False)

X = torch.ones(size=(1,3,8,8))
Y = conv2d(X)
Y.shape

torch.Size([1, 2, 8, 8])

- 위는 3개의 input channels, 2개의 output channels로 구현된 2D convolutional layer입니다. 
- 아래와 같이 convolutional layer의 kernel의 shape 및 값들을 출력하여 확인해 볼 수 있습니다.

In [5]:
print(conv2d.weight.data.shape)
print(conv2d.weight.data)

torch.Size([2, 3, 3, 3])
tensor([[[[-0.1731, -0.1180, -0.1183],
          [ 0.0219, -0.0671,  0.0985],
          [-0.1054, -0.0471,  0.0923]],

         [[-0.0237,  0.0507,  0.1341],
          [ 0.0858, -0.1690,  0.0830],
          [ 0.1548,  0.0453, -0.1069]],

         [[-0.1397, -0.0635,  0.0776],
          [-0.1821, -0.1640,  0.0089],
          [-0.0371,  0.0962,  0.0263]]],


        [[[-0.0013, -0.0108,  0.1307],
          [ 0.1496,  0.1721, -0.1839],
          [ 0.1135,  0.0179, -0.1702]],

         [[-0.0702,  0.1741, -0.0355],
          [-0.0506, -0.0813, -0.0556],
          [ 0.0076, -0.0715, -0.0833]],

         [[-0.0859, -0.0407,  0.1210],
          [ 0.1460,  0.1014, -0.1023],
          [ 0.0979,  0.1494,  0.0855]]]])


- 1x1 convolutional layer에 대해서도 살펴보겠습니다. 이는 kernel size를 1로 합니다.
- 16개의 채널을 입력으로 받아 1x1 convolution을 통해 1개의 채널로 줄여주는 것을 확인해 봅시다.

In [6]:
conv2d = nn.Conv2d(16,1,1,1,1,bias=False)
#conv2d = nn.Conv2d(in_channels=3,out_channels=2,kernel_size=3,stride=1,padding=1,bias=False)

X = torch.ones(size=(1,16,8,8))
Y = conv2d(X)
Y.shape

torch.Size([1, 1, 10, 10])

- 우리는 1x1 convolution을 통해 단지 16개의 파라미터를 이용하여 data를 1/16로 압축한 것을 확인할 수 있습니다.

In [7]:
print(conv2d.weight.data.shape)
print(conv2d.weight.data)

torch.Size([1, 16, 1, 1])
tensor([[[[ 0.0927]],

         [[-0.2428]],

         [[ 0.0087]],

         [[-0.2160]],

         [[ 0.0592]],

         [[-0.0105]],

         [[ 0.2484]],

         [[ 0.0165]],

         [[-0.1201]],

         [[-0.0570]],

         [[ 0.0412]],

         [[-0.0298]],

         [[-0.0811]],

         [[ 0.2303]],

         [[ 0.2226]],

         [[-0.1747]]]])


# 6.5. Pooling

- Pooling은 data를 축소하는 방법으로 보통 convolutional layer가 반복됨에 따라 data의 크기를 줄임과 동시에 점점 receptive field의 크기를 늘려주는 효과도 만들어 줍니다. 또한, 제한된 메모리크기에 대해서 채널의 수를 계속 늘려주는 것도 가능하게 합니다.
- 우리는 마지막 convolutional layer에서는 전체 image에 대한 feature를 효과적으로 한 번에 묶을 수 있는 것이 좋을 것입니다. 
- 많은 convolutional neural network에서 pooling은 점점 data의 크기를 축소시킴으로써, 전체 image에 대해 효과적으로 추출된 feature들을 한꺼번에 다음 fully connected layer의 입력으로 넣을 수 있도록 해줍니다.

In [8]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
print(X)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])


- 0부터 15까지의 숫자를 4x4 matrix로 만들어 X로 정의해 보았습니다. 이를 통해 실제 pooling이 어떤 역할을 하는 지 살펴보겠습니다.

In [9]:
pool2d = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2),padding=(0,0))
Y = pool2d(X)
Y

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

- nn.Conv2d와 마찬가지로, 파라미터의 이름을 명시하지 않고 정해진 순서대로 숫자를 넣어 사용해도 괜찮습니다.

In [10]:
pool2d = nn.MaxPool2d(2,2,0)
Y = pool2d(X)
Y

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

- Pooling은 각각의 채널에 대해서 수행됩니다. 따라서, 채널의 수를 바꾸지 않습니다.
- (경우에 따라서는 채널의 축으로 pooling을 하는 것도 물론 가능합니다.)

In [11]:
X2 = torch.cat((X,X+1),dim=1)
print('X.shape: ',X.shape)
print('X2.shape: ', X2.shape)
print('X2 = ',X2)

X.shape:  torch.Size([1, 1, 4, 4])
X2.shape:  torch.Size([1, 2, 4, 4])
X2 =  tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


In [12]:
pool2d = nn.MaxPool2d(2)
Y2 = pool2d(X2)
Y2

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

- 이해가 잘 되지 않을 때에는 print를 적극적으로 활용하도록 합니다.

In [13]:
print(pool2d.stride)
print(pool2d.padding)
print(pool2d.kernel_size)

2
0
2
